# Apache Hudi Core Conceptions (1) - Data Preparation

*Author: [Laurence Geng](https://laurence.blog.csdn.net) @ [https://laurence.blog.csdn.net](https://laurence.blog.csdn.net)*

Note: 

1. The public dataset amazon-reviews-pds is accessible on aws global regions, for China regions or non aws users, you can download it with S3 clients to local. 

2. The recommended configuration for Spark cluster is: 32 vCore，120GB or higher, the master node must have 100GB+ space.

## 1. Configuration

In [1]:
%env S3_BUCKET=apache-hudi-core-conceptions

env: S3_BUCKET=apache-hudi-core-conceptions


In [2]:
%%sql
set S3_BUCKET=apache-hudi-core-conceptions

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
95,application_1678096020253_0186,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [3]:
%%sh
aws s3 mb s3://${S3_BUCKET}

make_bucket: apache-hudi-core-conceptions


## 1. Create Raw Data Table

In [4]:
%%sql
drop table if exists all_reviews

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [5]:
%%sql
create external table if not exists all_reviews(
    marketplace string, 
    customer_id string, 
    review_id string, 
    product_id string, 
    product_parent string, 
    product_title string, 
    star_rating int, 
    helpful_votes int, 
    total_votes int, 
    vine string, 
    verified_purchase string, 
    review_headline string, 
    review_body string, 
    review_date date, 
    year int,
    product_category string
)
PARTITIONED BY (product_category)
STORED AS PARQUET
LOCATION 's3://amazon-reviews-pds/parquet/';

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [6]:
%%sql
msck repair table all_reviews;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [7]:
%%sql
select * from all_reviews limit 5;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [8]:
%%sql
select year, count(1) as count1, concat(cast(round((count(1) / 10000),0) as int), 'W') as count2 from all_reviews group by year order by year asc;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

## 2. Create Pruned Data Table

In [9]:
%%sh
aws s3 rm s3://${S3_BUCKET}/reviews --recursive &>/dev/null

In [10]:
%%sql
drop table if exists reviews

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [11]:
%%sql
create table reviews
using parquet
partitioned by (year)
location 's3://${S3_BUCKET}/reviews'
as select
    uuid() as review_id, 
    star_rating, 
    concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid()) as review_body, 
    review_date, 
    year
from
    all_reviews

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [12]:
%%sql
select review_id from reviews group by review_id having count(1) >= 2;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [13]:
%%sql
select count(1) from reviews;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [14]:
%%sql
select * from reviews limit 5;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [15]:
%%sql
select year, count(1) as count1, concat(cast(round((count(1) / 10000),0) as int), 'W') as count2 from reviews group by year order by year asc;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [16]:
%%sh
total_size=$(aws s3 ls --summarize --recursive "s3://${S3_BUCKET}/reviews/" | sed -nE "s/\s+Total\sSize\:\s+(\d*)/\1/p")
echo -e "Total Size:\t[ ${total_size} ]"
echo -e "Total Records:\t[ 160796570 ]"
avg_size=$((total_size/160796570))
echo -e "Average Size:\t[ ${avg_size} ] Bytes"

Total Size:	[ 40167017791 ]
Total Records:	[ 160796570 ]
Average Size:	[ 249 ] Bytes


Note: Hudi talbes' schema is different from reviews table, the average record size of hudi table is [ 175 ] bytes not [ 249 ] Bytes!!
This is a question, becuase hudi table has 5 + 2 = 7 extra fields, but it is smaller (not compressed)!